In [29]:

import urllib.request

api_key = "api_key"


In [31]:
url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Central%20Park%20New%20York/2023-09-20/2023-10-01?unitGroup=us&maxDistance=1000&include=days&key={api_key}&elevationDifference=1&contentType=csv'
destination = 'Data/infer_temp.csv'

urllib.request.urlretrieve(url, destination)

('Data/infer_temp.csv', <http.client.HTTPMessage at 0x28066d210>)

In [32]:
import pandas as pd
import numpy as np
import pickle

In [33]:
df = pd.read_csv(destination)
# df.head()

In [34]:
df.tail()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
7,"Central Park, New York, NY 10021, United States",2023-09-27,65.9,52.1,59.0,65.9,52.1,59.0,48.2,68.3,...,16.3,6,10,2023-09-27T06:48:10,2023-09-27T18:45:07,0.44,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,remote
8,"Central Park, New York, NY 10021, United States",2023-09-28,65.0,54.9,60.4,65.0,54.9,60.4,54.1,80.0,...,5.0,2,10,2023-09-28T06:49:11,2023-09-28T18:43:26,0.47,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,remote
9,"Central Park, New York, NY 10021, United States",2023-09-29,62.0,58.9,60.3,62.0,58.9,60.3,58.5,93.9,...,0.9,1,10,2023-09-29T06:50:11,2023-09-29T18:41:46,0.50,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,remote
10,"Central Park, New York, NY 10021, United States",2023-09-30,65.9,58.9,61.7,65.9,58.9,61.7,57.0,85.0,...,6.9,3,10,2023-09-30T06:51:12,2023-09-30T18:40:05,0.54,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,remote
11,"Central Park, New York, NY 10021, United States",2023-10-01,74.9,58.9,67.5,74.9,58.9,67.5,55.4,66.8,...,17.8,7,10,2023-10-01T06:52:13,2023-10-01T18:38:25,0.57,Clear,Clear conditions throughout the day.,clear-day,remote


In [35]:
file = open("models/lin_reg.pkl",'rb')
model = pickle.load(file)
file.close()

In [36]:
def give_next24hrs_pred(model, df):

    df.rename(columns = {'datetime':'DATE', 'tempmax':'TMAX', 'tempmin':'TMIN'}, inplace = True)

    df = df[['DATE', 'TMAX', 'TMIN']]

    df['TMAX_prev7'] = df['TMAX'].rolling(7).max()
    df['TMAX_prev3'] = df['TMAX'].rolling(3).max()
    df['TMAX_prev2'] = df['TMAX'].rolling(2).max()

    df['TMAX_prev1'] = df['TMAX'].rolling(1).max()

    df['TMIN_prev3'] = df['TMIN'].rolling(3).max()
    df['TMIN_prev2'] = df['TMIN'].rolling(2).max()
    df['TMIN_prev1'] = df['TMIN'].rolling(1).max()

    df = df.loc[7:, :]

    df['DATE'] = pd.to_datetime(df['DATE'])

    df['day'] = df['DATE'].dt.day
    df['month'] = df['DATE'].dt.month

    df.sort_values(by = ['DATE'], inplace = True)
    df.set_index("DATE", inplace = True)

    df.fillna(-1, inplace = True)

    df.drop(['TMIN', 'TMAX'], axis = 1, inplace = True)

    print(df)

    return model.predict(df)




In [37]:
preds = give_next24hrs_pred(model, df)

            TMAX_prev7  TMAX_prev3  TMAX_prev2  TMAX_prev1  TMIN_prev3  \
DATE                                                                     
2023-09-27        72.9        65.9        65.9        65.9        58.9   
2023-09-28        71.0        65.9        65.9        65.0        58.0   
2023-09-29        65.9        65.9        65.0        62.0        58.9   
2023-09-30        65.9        65.9        65.9        65.9        58.9   
2023-10-01        74.9        74.9        74.9        74.9        58.9   

            TMIN_prev2  TMIN_prev1  day  month  
DATE                                            
2023-09-27        58.0        52.1   27      9  
2023-09-28        54.9        54.9   28      9  
2023-09-29        58.9        58.9   29      9  
2023-09-30        58.9        58.9   30      9  
2023-10-01        58.9        58.9    1     10  


/var/folders/x5/1285xgcd3vl1lvxv_nmsdqtc0000gn/T/ipykernel_3135/509194500.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TMAX_prev7'] = df['TMAX'].rolling(7).max()
/var/folders/x5/1285xgcd3vl1lvxv_nmsdqtc0000gn/T/ipykernel_3135/509194500.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TMAX_prev3'] = df['TMAX'].rolling(3).max()
/var/folders/x5/1285xgcd3vl1lvxv_nmsdqtc0000gn/T/ipykernel_3135/509194500.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [38]:
preds[-1]

76.854965

In [39]:
print('Prediction next day temperature : ', preds[-1])

Prediction next day temperature :  76.854965
